**Import necessari**

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, UpSampling2D
from tensorflow.keras import models
from tensorflow.keras import losses
from tensorflow.keras import optimizers
import numpy as np
import random
from cv2 import GaussianBlur

**Funzioni utilizzate**

In [ ]:
def getDataset():
    (x_train, _), (x_test, _) = tf.keras.datasets.cifar10.load_data()
    normalize = lambda x: x/x.max()
    x_train = normalize(x_train)
    x_test = normalize(x_test)
    createBlurred = lambda arr: np.array([GaussianBlur(x, (5,5), ((random.random()*3)+1)) + (0.02*np.random.normal(0,1, (32, 32, 3))) for x in arr])
    return (createBlurred(x_train), x_train), (createBlurred(x_test), x_test)

**Definizione del modello**

In [ ]:
# La funzione UNet definisce l'architettura della rete neurale U-Net.
# Prende in input una forma di input (input_shape) e restituisce un modello Keras

def UNet(input_shape):
    keras.backend.clear_session()
    inputs = Input(input_shape)

    # Encoder
    # L'encoder inizia con due blocchi di convoluzione consecutivi (conv1) seguiti
    # da un max pooling (pool1). I parametri utilizzati per le convoluzioni sono:
    # - 16 filtri
    # - Dimensione del kernel 3x3
    # - Funzione di attivazione ReLU
    # - Padding "same" per mantenere le dimensioni
    # - Inizializzazione dei pesi con l'inizializzatore "he_normal"
    conv1 = Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    # Il secondo blocco (conv2) segue lo stesso schema del primo, con 32 filtri
    conv2 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Il terzo blocco (conv3) ha 64 filtri
    conv3 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # Il quarto blocco (conv4) ha ancora 64 filtri e
    # un dropout del 25% dopo la seconda convoluzione
    conv4 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.25)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    # Bottleneck
    # Il blocco del bottleneck (conv5) ha 64 filtri ed è seguito da un dropout del 25%
    conv5 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = Dropout(0.25)(conv5)

    # Decoder
    # Il primo blocco di decodifica (up6) utilizza un'operazione di upsampling
    # per aumentare le dimensioni dell'immagine seguita da una convoluzione con 64 filtri.
    # Il risultato viene concatenato con il quarto blocco di convoluzione (drop4) tramite l'operazione concatenate.
    # Seguono due convoluzioni aggiuntive (conv6) con 64 filtri ciascuna.
    up6 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    # Il secondo blocco di decodifica (up7) utilizza un'operazione di upsampling
    # seguita da una convoluzione con 32 filtri. Il risultato viene concatenato
    # con il terzo blocco di convoluzione (conv3).
    # Vengono poi applicate due convoluzioni (conv7) con 32 filtri ciascuna.
    up7 = Conv2D(32, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    # Il terzo blocco di decodifica (up8) utilizza un'operazione di upsampling
    # seguita da una convoluzione con 16 filtri. Il risultato viene concatenato
    # con il secondo blocco di convoluzione (conv2).
    # Seguono due convoluzioni (conv8) con 16 filtri ciascuna.
    up8 = Conv2D(16, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

    # Il quarto blocco di decodifica (up9) utilizza un'operazione di upsampling
    # seguita da una convoluzione con 8 filtri. Il risultato viene concatenato
    # con il primo blocco di convoluzione (conv1).
    # Seguono due convoluzioni (conv9) con 8 filtri ciascuna.
    up9 = Conv2D(8, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(8, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = Conv2D(8, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)

    # Output
    # L'output finale è ottenuto applicando una convoluzione con 3 filtri e una
    # dimensione del kernel di 3x3. Successivamente, viene applicata una convoluzione
    # con 1 filtro e una dimensione del kernel di 1x1 per ottenere il risultato finale dell'immagine ricostruita.
    conv9 = Conv2D(3, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    outputs = layers.Conv2D(3, 1, activation='sigmoid')(conv9)

    model = keras.Model(inputs=inputs, outputs=outputs, name='UNet')
    model.summary()

    return model

**Fase di training del modello**

In [ ]:
# Carico e pre-elaboro i dati
(x_train, y_train), (x_test, y_test) = getDataset()

# Suddivido il dataset di addestramento in train e validation set
val_split = 0.05
val_samples = int(val_split * len(x_train))
X_val = x_train[-val_samples:]
y_val = y_train[-val_samples:]
x_train = x_train[:val_samples]
y_train = y_train[:val_samples]

model = UNet((32, 32, 3))

EPOCHS = 100
BATCH_SIZE = 32  # Riduci la dimensione del batch
callbacks=[keras.callbacks.ModelCheckpoint('pesi_rete.h5',save_best_only=True)]

# Aggiungo un learning rate scheduler
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

model.compile(optimizer=optimizers.Adam(),
              loss=losses.MeanSquaredError(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    validation_data=(X_val, y_val),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    callbacks=[callbacks],
                    verbose=1)

(50000, 32, 32, 3)
(50000, 32, 32, 3)
Model: "UNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 32, 32, 16)   2320        ['conv2d[0][0]']                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 16, 16, 16)   0           ['conv2d_1[0][0]']               
                                                         

In [ ]:
model = tf.keras.models.load_model('pesi_rete.h5')


# Carica e pre-elabora i dati
(x_train, y_train), (x_test, y_test) = getDataset()

# Suddividi il dataset di addestramento in train e validation set
val_split = 0.05
val_samples = int(val_split * len(x_train))
X_val = x_train[-val_samples:]
y_val = y_train[-val_samples:]
x_train = x_train[:val_samples]
y_train = y_train[:val_samples]

EPOCHS = 50
BATCH_SIZE = 16  # Riduci la dimensione del batch
callbacks=[keras.callbacks.ModelCheckpoint('pesi_rete_2.h5',save_best_only=True)]

# Aggiungi un learning rate scheduler
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=10)

model.compile(optimizer=optimizers.Adam(),
              loss=losses.MeanSquaredError(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    validation_data=(X_val, y_val),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    callbacks=[callbacks, lr_scheduler],
                    verbose=1)

Epoch 1/50
157/157 [==============================] - 41s 231ms/step - loss: 0.0042 - accuracy: 0.7788 - val_loss: 0.0036 - val_accuracy: 0.8160 - lr: 0.0010
Epoch 2/50
157/157 [==============================] - 31s 200ms/step - loss: 0.0033 - accuracy: 0.8172 - val_loss: 0.0035 - val_accuracy: 0.8081 - lr: 0.0010
Epoch 3/50
157/157 [==============================] - 35s 225ms/step - loss: 0.0033 - accuracy: 0.8173 - val_loss: 0.0034 - val_accuracy: 0.8181 - lr: 0.0010
Epoch 4/50
157/157 [==============================] - 35s 225ms/step - loss: 0.0032 - accuracy: 0.8171 - val_loss: 0.0034 - val_accuracy: 0.8239 - lr: 0.0010
Epoch 5/50
157/157 [==============================] - 30s 193ms/step - loss: 0.0032 - accuracy: 0.8184 - val_loss: 0.0036 - val_accuracy: 0.8220 - lr: 0.0010
Epoch 6/50
157/157 [==============================] - 35s 225ms/step - loss: 0.0034 - accuracy: 0.8182 - val_loss: 0.0034 - val_accuracy: 0.8221 - lr: 0.0010
Epoch 7/50
157/157 [==============================] 